We started our journey by analysing our dataset ODIR-2019 
- https://www.kaggle.com/datasets/andrewmvd/ocular-disease-recognition-odir5k/data
- https://odir2019.grand-challenge.org/dataset/

From the dataset kaggle page:

> Ocular Disease Intelligent Recognition (ODIR) is a structured ophthalmic database of 5,000 patients with age, 
> color fundus photographs from left and right eyes and doctors' diagnostic keywords from doctors.

> This dataset is meant to represent ‘‘real-life’’ set of patient information collected by Shanggong Medical Technology Co., Ltd. from different hospitals/medical centers in China. In these institutions, fundus images are captured by various cameras in the market, such as Canon, Zeiss and Kowa, resulting into varied image resolutions. Annotations were labeled by trained human readers with quality control management. They classify patient into eight labels including:

> Normal (N),
> Diabetes (D),
> Glaucoma (G),
> Cataract (C),
> Age related Macular Degeneration (A),
> Hypertension (H),
> Pathological Myopia (M),
> Other diseases/abnormalities (O)

Also from the ODIR-2019 grand challenge website:

> The 5,000 patients in this challenge are divided into training, off-site testing and on-site testing subsets. Almost 4,000 cases are used in training stage while  others are for testing stages (off-site and on-site). Table 2 shows the distribution of case number with respect to eight labels in different stages. **Note: one patient may contains one or multiple labels.**

We realise that we have multi-label classification dataset

In [8]:
import numpy as np
import pandas as pd
from PIL import Image
data_set_path = 'ODIR-2019/dataset/full_df.csv'
data_set_img_path = 'ODIR-2019/dataset/preprocessed_images/'

Let us explore the dataset csv or index file to understand our data

In [9]:
df = pd.read_csv(data_set_path)
df.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6392 entries, 0 to 6391
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ID                         6392 non-null   int64 
 1   Patient Age                6392 non-null   int64 
 2   Patient Sex                6392 non-null   object
 3   Left-Fundus                6392 non-null   object
 4   Right-Fundus               6392 non-null   object
 5   Left-Diagnostic Keywords   6392 non-null   object
 6   Right-Diagnostic Keywords  6392 non-null   object
 7   N                          6392 non-null   int64 
 8   D                          6392 non-null   int64 
 9   G                          6392 non-null   int64 
 10  C                          6392 non-null   int64 
 11  A                          6392 non-null   int64 
 12  H                          6392 non-null   int64 
 13  M                          6392 non-null   int64 
 14  O       

Let's convert label to python array

We need to simplify this table given there is information for both left and right eyes for the same patient ID in the same row.
Let's explore by viewing patient ID 0

In [19]:
df[df.ID==0]

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['C'],"[0, 0, 0, 1, 0, 0, 0, 0]",0_left.jpg


The 2 rows for patient with ID 0 shows how the column relate to the each eye image. We should use filename as unique ID for each fundus file image, labels indicate whether patient has eye problem like cataract or normal. Let's check what are possible values for labels column.

In [11]:
df.labels.value_counts()

labels
['N']    2873
['D']    1608
['O']     708
['C']     293
['G']     284
['A']     266
['M']     232
['H']     128
Name: count, dtype: int64

From label value, we realize that each fundus image only have 1 specific label. Hence, this is still a multi-class classification problem which make our problem simpler as we just need to assign 1 label to each image.

Let's simplify our data and remove factor like Age and Gender from our data.

In [12]:
from ast import literal_eval
df2= df[['filename', 'labels', 'ID']].copy()
df2['labels'] = df2['labels'].apply(literal_eval)
df2['labels'] = df2['labels'].apply(lambda x: x[0])
df2.rename(columns={'labels':'label'}, inplace=True)
map_label = {
    'N': 'Normal',
    'D': 'Diabetes',
    'G': 'Glaucoma',
    'C': 'Cataract',
    'A': 'Ageing',
    'H': 'Hypertension',
    'M': 'Myopia',
    'O': 'Other diseases'
}
df2['class'] = df2['label'].map(map_label)
df2.head()

,filename,label,ID,class
0,0_right.jpg,N,0,Normal
1,1_right.jpg,N,1,Normal
2,2_right.jpg,D,2,Diabetes
3,4_right.jpg,D,4,Diabetes
4,5_right.jpg,D,5,Diabetes


Let us also check image sizes

In [18]:
def get_image_dimensions(image_path):
    """
    Gets the width and height of an image without loading the full raster data.

    Args:
        image_path (str): The path to the image file.

    Returns:
        tuple: A tuple containing (width, height) in pixels.
    """
    try:
        with Image.open(image_path) as img:
            width, height = img.size
            return width, height
    except IOError:
        print(f"Cannot open image file: {image_path}")
        return None, None
    
for index, row in df2.iterrows():
    image_path = data_set_img_path + row['filename']
    width, height = get_image_dimensions(image_path)
    index = hash(index)
    df2.at[index, 'width'] = width
    df2.at[index, 'height'] = height
    df2.at[index, 'wxh']  = f"{width}x{height}"

df2.head()

,filename,label,ID,class,width,height,wxh,"(0, width)","(1, width)","(2, width)",...,"(6382, width)","(6383, width)","(6384, width)","(6385, width)","(6386, width)","(6387, width)","(6388, width)","(6389, width)","(6390, width)","(6391, width)"
0,0_right.jpg,N,0,Normal,512.0,512.0,512x512,512,512,512,...,512,512,512,512,512,512,512,512,512,512
1,1_right.jpg,N,1,Normal,512.0,512.0,512x512,512,512,512,...,512,512,512,512,512,512,512,512,512,512
2,2_right.jpg,D,2,Diabetes,512.0,512.0,512x512,512,512,512,...,512,512,512,512,512,512,512,512,512,512
3,4_right.jpg,D,4,Diabetes,512.0,512.0,512x512,512,512,512,...,512,512,512,512,512,512,512,512,512,512
4,5_right.jpg,D,5,Diabetes,512.0,512.0,512x512,512,512,512,...,512,512,512,512,512,512,512,512,512,512


In [25]:
df2.wxh.value_counts()

wxh
512x512    6392
Name: count, dtype: int64

In [23]:
import os 
import shutil
yolo_datapath = "ODIR-2019/yolo"
os.makedirs(yolo_datapath, exist_ok=True)
map_dir = { k:os.path.join(yolo_datapath, k) for k in map_label.keys() }
for label, path in map_dir.items():
    os.makedirs(path, exist_ok=True)
    for file_name in df2[df2.label==label]['filename']:
        shutil.copy2(os.path.join(data_set_img_path, file_name), os.path.join(path, file_name))